Import modules

In [ ]:
import importlib
from System.paths import *
from OMM_interface.openmm import *
import os
import shutil
from VMD import VMD
import importlib
import os
import numpy as np
from pathlib import Path
import re
from ASE_interface.ase_calculation import CP2K
from Coord_Toolz.mdanalysis import MDA_reader
from Direct_cp2k_calculation.direct_cp2k import Direct_Calculator
from System.paths import Paths
from System.system import Molecular_system
from ASE_interface.ase_calculation import ASE_system
from Parametrization.parametrization import Parametrization
from Parametrization.optimizers import Optimizer
from System.system import write_single_pdb
from OMM_interface.openmm import OpenMM_system

Configure Paths

In [ ]:
paths = Paths()

paths.working_dir = '../methylphosphate/force_matching/test/'
paths.project_name = 'mp0mp0'

paths.mm_top_file = 'smallbox.psf'
paths.mm_top_file_path = '/home/ac127777/Documents/methylphosphate/cp2k_geom_opts/mp0mp0/frame0/'
paths.mm_crd_file = 'smallbox.pdb'
paths.mm_crd_file_path = paths.mm_top_file_path
paths.mm_str_file = 'mp0.str'
paths.mm_str_file_path = '../methylphosphate/force_matching/test/'
paths.mm_traj_file = 'optcoords.xyz'
paths.mm_traj_file_path = '/home/ac127777/Documents/methylphosphate/cp2k_geom_opts/mp0mp0/'

mm_mol1_crd_file = 'optpos_phos1.pdb'
mm_mol1_crd_file_path = paths.mm_top_file_path
mm_mol1_top_file = 'optpos_phos1.psf'
mm_mol1_top_file_path = paths.mm_top_file_path
mm_mol2_crd_file = 'optpos_phos2.pdb'
mm_mol2_crd_file_path = paths.mm_top_file_path
mm_mol2_top_file = 'optpos_phos2.psf'
mm_mol2_top_file_path = paths.mm_top_file_path

paths.set()

Set up System.system.Molecular_system

In [ ]:
Molsys = Molecular_system('2_solvent', 'net_properties', 'forces')
Molsys.paths = paths

Set up Coord_Toolz.mdanalysis.MDA_reader object for both molecules dissolved, one molecule dissolved, and the other molecule dissolved

In [ ]:
MDR = MDA_reader()
MDR.set_traj_input(paths.crd_pdb, paths.traj)

In [ ]:
MDR.mol1 = MDR.delete_one_molecule('not resid 2')
MDR.mol2 = MDR.delete_one_molecule('not resid 1')

Fill Molecular_system storage

In [ ]:
Molsys.set_ini_coords(MDR)

Set up OpenMM system(s) and link them to Molecular_system

first do entire system

In [ ]:
omm_sys_all = OpenMM_system(topology_format='CHARMM', top_file=paths.mm_top, crd_format='PDB',
                        crd_file=paths.mm_crd, charmm_param_file=paths.mm_stream, pbc=True,
                        cell=[1.6, 1.6, 1.6], angles=[90, 90, 90])

In [ ]:
omm_sys_all.create_system_params['nonbondedCutoff'] = 0.8 * unit.nanometer
omm_sys_all.create_system_params['constraints'] = app.HBonds
omm_sys_all.import_molecular_system()
omm_sys_all.set_platform()
omm_sys_all.create_openmm_system()
omm_sys_all.set_openmm_context()

In [ ]:
Molsys.openmm_sys['all'] = omm_sys_all

then for molecule 1 

In [ ]:
omm_sys_mol1 = OpenMM_system(topology_format='CHARMM', top_file=paths.mm_mol1_top, crd_format='PDB',
                        crd_file=paths.mm_mol1_crd, charmm_param_file=paths.mm_stream, pbc=True,
                        cell=[1.6, 1.6, 1.6], angles=[90, 90, 90])

In [ ]:
omm_sys_mol1.create_system_params['nonbondedCutoff'] = 0.8 * unit.nanometer
omm_sys_mol1.create_system_params['constraints'] = app.HBonds
omm_sys_mol1.import_molecular_system()
omm_sys_mol1.set_platform()
omm_sys_mol1.create_openmm_system()
omm_sys_mol1.set_openmm_context()

In [ ]:
Molsys.openmm_sys['all'] = omm_sys_mol1

...and molecule 2

In [ ]:
omm_sys_mol2 = OpenMM_system(topology_format='CHARMM', top_file=paths.mm_mol2_top, crd_format='PDB',
                        crd_file=paths.mm_mol2_crd, charmm_param_file=paths.mm_stream, pbc=True,
                        cell=[1.6, 1.6, 1.6], angles=[90, 90, 90])

In [ ]:
omm_sys_mol2.create_system_params['nonbondedCutoff'] = 0.8 * unit.nanometer
omm_sys_mol2.create_system_params['constraints'] = app.HBonds
omm_sys_mol2.import_molecular_system()
omm_sys_mol2.set_platform()
omm_sys_mol2.create_openmm_system()
omm_sys_mol2.set_openmm_context()

In [ ]:
Molsys.openmm_sys['all'] = omm_sys_mol2

...not extract ff and decide on type of reference (dft) data....